# Control Knowledge

This notebook contains analysis of using control knowledge in FondASP.

Domains that were tested are as below

| Domain             | Plain                   | Control Knowledge                       | Type of Knowledge                 |
|--------------------|-------------------------|-----------------------------------------|-----------------------------------|
| Acrobatics         | asp2-backbone-4h-single | asp2-4h-undo,asp2-4h-kb-single          | undo, dead end state (broken leg) |
| Blocksworld-ipc    | asp2-backbone-4h        | asp2-4h-undo                            | undo                              |
| First Responders   | asp2-backbone-4h        | asp2-4h-undo                            | undo                              |
| Islands            | asp2-backbone-4h-single | asp2-4h-undo-single                     | undo                              |
| Miner              | asp2-backbone-4h-single | asp2-4h-undo-single, asp2-4h-kb2-single | undo, dead end state (Alive)      |
| Spiky Tireworld    | asp2-backbone-4h-single | asp2-4h-undo-single                     | undo                              |
| Tireworld Truck    | asp2-backbone-4h-single | asp2-4h-undo-single                     | undo                              |
| Tireworld Triangle | asp2-backbone-4h        | asp2-4h-kb                              | dead end state (has spare)        |

In [4]:
relevant_configs = {
    "acrobatics": ["asp1-4h", "asp1-4h-undo"],
    "blocksworld-ipc08": ["asp1-4h", "asp1-4h-undo"],
    "first-responders-ipc08": ["asp1-4h", "asp1-4h-undo"],
    "islands": ["asp1-4h", "asp1-4h-undo"],
    "miner": ["asp1-4h", "asp1-4h-undo", "asp1-4h-kb"],
    "spiky-tireworld": ["asp1-4h", "asp1-4h-undo"],
    "tireworld-truck": ["asp1-4h", "asp1-4h-undo"],
    "triangle-tireworld": ["asp1-4h", "asp1-4h-kb"]
    }

In [5]:
import pandas as pd
import numpy as np

In [6]:
LOC = "~/Work/Data/FondAsp"
df = pd.read_csv(f"{LOC}/results.csv")
df.head()

,domain,instance,planner,type,solved,SAT,time,memory,timeout,memoryout,policysize
0,acrobatics,p06,asp1-4h,asp,False,-1,14400.383668,2158.449219,True,False,36
1,acrobatics,p06,paladinus-4h,paladinus,True,True,3.153158,82.828125,False,False,128
2,acrobatics,p06,minisat-4h,fondsat,False,-1,14400.285015,1983.265625,True,False,-1
3,acrobatics,p06,glucose-4h,fondsat,False,-1,14399.936225,610.523438,True,False,-1
4,acrobatics,p06,asp2-backbone-4h,asp,False,-1,14400.779252,2543.703125,True,False,38


In [12]:
df_sat = pd.read_csv(f"{LOC}/sat_results.csv")
df_sat = df_sat.loc[:, ~df_sat.columns.str.contains('Unnamed')]

df_unsat = pd.read_csv(f"{LOC}/unsat_results.csv")
df_unsat = df_unsat.loc[:, ~df_unsat.columns.str.contains('Unnamed')]

df_sat_counts = pd.read_csv(f"{LOC}/counts.csv", index_col="domain")
df_sat_counts

df_sat.head()

,domain,instance,planner,type,solved,SAT,time,memory,timeout,memoryout,policysize
0,acrobatics,p06,asp1-4h,asp,False,-1,14400.383668,2158.449219,True,False,36
1,acrobatics,p06,paladinus-4h,paladinus,True,True,3.153158,82.828125,False,False,128
2,acrobatics,p06,minisat-4h,fondsat,False,-1,14400.285015,1983.265625,True,False,-1
3,acrobatics,p06,glucose-4h,fondsat,False,-1,14399.936225,610.523438,True,False,-1
4,acrobatics,p06,asp2-backbone-4h,asp,False,-1,14400.779252,2543.703125,True,False,38


In [65]:
new_domains = ["miner", "doors", "islands", "spiky-tireworld", "tireworld-truck"]
df_sat_counts.query(f"domain not in @new_domains")["total_count"].sum()
df_sat_counts

,total_count
domain,
acrobatics,8
beam-walk,11
blocksworld-ipc08,30
blocksworld-new,40
chain-of-rooms,10
doors,15
earth_observation,40
elevators,15
faults-ipc08,55


## Count the number of solved instances

In [32]:
minimum_amount = 10
df_asp_solved = df_sat.query("solved == True and planner.str.contains('asp')")
df_asp_counts = df_asp_solved.groupby(["domain", "planner"]).count()[["instance"]]
df_asp_counts.rename(columns={"instance": "solved"}, inplace=True)
df_asp_counts.reset_index(inplace=True)
df_asp_counts.query(f"solved>=@minimum_amount")

,domain,planner,solved
8,blocksworld-ipc08,asp1-4h,10
9,blocksworld-ipc08,asp2-4h-undo,10
10,blocksworld-ipc08,asp2-backbone-4h,10
11,blocksworld-ipc08,asp2-reg-undo-4h,10
16,doors,asp1-4h,15
17,doors,asp1-reg-4h,14
18,doors,asp2-backbone-4h,15
19,doors,asp2-reg-4h,15
20,earth_observation,asp1-4h,10
21,earth_observation,asp2-backbone-4h,10


In [35]:
asp_planners = ["asp2-reg-4h", "asp2-reg-kb-4h", "asp2-reg-undo-4h"]
domains = df_asp_counts.query(f"solved>=@minimum_amount").domain.unique()
df_asp_solved = df_sat.query(f"solved == True and planner in @asp_planners and domain in @domains")
df_asp_solved

,domain,instance,planner,type,solved,SAT,time,memory,timeout,memoryout,policysize
104,blocksworld-ipc08,p06,asp2-reg-undo-4h,asp,True,True,10.561045,0.812500,False,False,11
112,blocksworld-ipc08,p01,asp2-reg-undo-4h,asp,True,True,2.552596,1.292969,False,False,9
148,blocksworld-ipc08,p03,asp2-reg-undo-4h,asp,True,True,3.156630,1.246094,False,False,10
191,blocksworld-ipc08,p07,asp2-reg-undo-4h,asp,True,True,16.693293,1.089844,False,False,13
199,blocksworld-ipc08,p09,asp2-reg-undo-4h,asp,True,True,2.461418,0.910156,False,False,9
...,...,...,...,...,...,...,...,...,...,...,...
4155,tireworld-truck,p37,asp2-reg-undo-4h,asp,True,True,2.655086,1.093750,False,False,13
4164,tireworld-truck,p19,asp2-reg-4h,asp,True,True,2.642422,1.199219,False,False,14
4165,tireworld-truck,p19,asp2-reg-undo-4h,asp,True,True,2.878265,1.171875,False,False,14
4174,tireworld-truck,p04,asp2-reg-4h,asp,True,True,1.206287,1.023438,False,False,11


In [36]:
df_asp_solved.to_csv("ck_results.csv")

In [38]:
configs = {
    "blocksworld-ipc08": "asp2-reg-undo-4h",
    "tireworld-truck": "asp2-reg-undo-4h",
    "first-responders-ipc08": "asp2-reg-undo-4h",
    "islands": "asp2-reg-undo-4h",
    "miner": "asp2-reg-kb-4h",
    "spiky-tireworld": "asp2-reg-kb-4h"
}

In [61]:
selected_dfs = []
for _d, _p in configs.items():
    df_1 = df.query(f"domain == @_d and planner == @_p")
    df_1["planner"] = "CASP-KB"
    df_2 = df.query(f"domain == @_d and planner =='asp2-reg-4h'")
    df_2["planner"] = "CASP-Base"
    
    i_1 = df_1.query("domain==@_d")["instance"]
    i_2 = df_2.query("domain==@_d")["instance"]
    common = set(i_1).intersection(set(i_2))
    
    df_3 = df_1.query(f"instance in @common")
    df_4 = df_2.query(f"instance in @common")

    selected_dfs.append(df_3)
    selected_dfs.append(df_4)

    
    
df_selected = pd.concat(selected_dfs)
df_selected.to_csv(f"ck_result_plots.csv")

/tmp/ipykernel_569631/3907636651.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1["planner"] = "CASP-KB"
/tmp/ipykernel_569631/3907636651.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2["planner"] = "CASP-Base"


In [59]:
i_1 = df_selected.query("domain=='spiky-tireworld' and planner=='CASP-Base'")["instance"]
i_2 = df_selected.query("domain=='spiky-tireworld' and planner=='CASP-KB'")["instance"]
common = set(i_1).intersection(set(i_2))
df_selected.query(f"domain == 'spiky-tireworld' and instance in @common")

,domain,instance,planner,type,solved,SAT,time,memory,timeout,memoryout,policysize
3551,spiky-tireworld,p06,CASP-KB,asp,True,True,235.359791,365.378906,False,False,23
3562,spiky-tireworld,p01,CASP-KB,asp,True,True,53.229523,109.160156,False,False,23
3573,spiky-tireworld,p03,CASP-KB,asp,True,True,78.691598,267.089844,False,False,23
3584,spiky-tireworld,p07,CASP-KB,asp,True,True,255.742661,420.410156,False,False,23
3595,spiky-tireworld,p09,CASP-KB,asp,True,True,357.252640,502.074219,False,False,23
3606,spiky-tireworld,p08,CASP-KB,asp,True,True,398.404447,472.218750,False,False,23
3617,spiky-tireworld,p02,CASP-KB,asp,True,True,59.757721,219.097656,False,False,23
3637,spiky-tireworld,p05,CASP-KB,asp,True,True,155.834312,353.695312,False,False,23
3657,spiky-tireworld,p04,CASP-KB,asp,True,True,11.134973,1.187500,False,False,23
3549,spiky-tireworld,p06,CASP-Base,asp,True,True,4269.018172,358.890625,False,False,23


### Helper functions

In [14]:
def coverage(df_domain, df_counts):
    df_solved = df_domain.query("solved==True")
    df_solved_count = df_solved.groupby(["domain", "planner"]).count()[["instance"]].reset_index()
    df_solved_count.rename(columns={"instance": "count"}, inplace=True)
    df_coverage = df_solved_count.merge(df_counts, left_on="domain", right_on="domain", how="inner")
    df_coverage["coverage"] = df_coverage["count"]/df_coverage["total_count"]
    return df_coverage.pivot_table(index=["domain"], columns=["planner"], values=["coverage"])

def average_time_diff(df_domain, planner_1, planner_2):
    df_times = df_domain.query("solved==True").pivot_table(index=["domain", "instance"], columns=["planner"], values=["time"]).dropna()
    df_times.columns = ['_'.join(col) for col in df_times.columns.values]
    df_times["diff"] = 100*(df_times[f"time_{planner_1}"] - df_times[f"time_{planner_2}"])/df_times[f"time_{planner_1}"]
    display(df_times)
    return round(df_times[f"time_{planner_1}"].mean(),2),round(df_times[f"time_{planner_2}"].mean(),2), round(df_times['diff'].mean(),2)

def stats_kb(domain, planner_1, planner_2):
    configs = relevant_configs[domain]
    df_scenario = df_sat.query(f"domain==@domain and planner in @configs")
    df_c = coverage(df_scenario, df_sat_counts)
    p1_mean, p2_mean, percentage_diff = average_time_diff(df_scenario, planner_1, planner_2)
    
    return df_c, p1_mean, p2_mean, percentage_diff

## Scenario analysis on SAT instances

### Acrobatics

In [250]:
domain = "acrobatics"
configs = relevant_configs[domain]
df_acrobatics = df_sat.query(f"domain==@domain and planner in @configs")
df_acrobatics.head()

,domain,instance,planner,type,solved,SAT,time,memory,timeout,memoryout,policysize
3770,acrobatics,p05,asp1-4h,asp,False,-1,14400.361239,1339.824219,True,False,40
3775,acrobatics,p05,asp1-4h-undo,asp,False,-1,14400.342052,1376.441406,True,False,40
3782,acrobatics,p02,asp1-4h,asp,True,True,1.484706,1.042969,False,False,8
3787,acrobatics,p02,asp1-4h-undo,asp,True,True,1.377246,0.921875,False,False,8
3794,acrobatics,p03,asp1-4h,asp,True,True,9.833538,0.929688,False,False,16


In [251]:
coverage(df_acrobatics, df_sat_counts)

coverage             
planner     asp1-4h asp1-4h-undo
domain                          
acrobatics      0.5          0.5

In [252]:
#df_times = df_acrobatics.query("solved==True").pivot_table(index=["domain", "instance"], columns=["planner"], values=["time"])
#df_times.columns = ['_'.join(col) for col in df_times.columns.values]
#df_times["diff"] = 100*(df_times['time_asp2-backbone-4h-single'] - df_times['time_asp2-4h-kb-single'])/df_times['time_asp2-backbone-4h-single']
percentage_diff = average_time_diff(df_acrobatics, 'asp1-4h', 'asp1-4h-undo')
print(f"Average decrease in solve times by {percentage_diff}%")


time_asp1-4h  time_asp1-4h-undo       diff
domain     instance                                            
acrobatics p01           1.183176           1.270871  -7.411878
           p02           1.484706           1.377246   7.237799
           p03           9.833538           7.527270  23.453084
           p04        1517.849103         715.065836  52.889531

Average decrease in solve times by (382.59, 181.31, 19.04)%


In [253]:
df_c, p1_time, p2_time, improvement = stats_kb("acrobatics", 'asp1-4h', 'asp1-4h-undo')
display(df_c)
print(p1_time, p2_time, (p1_time-p2_time)/p1_time)
print(f"Average decrease in solve times by {improvement}%")

time_asp1-4h  time_asp1-4h-undo       diff
domain     instance                                            
acrobatics p01           1.183176           1.270871  -7.411878
           p02           1.484706           1.377246   7.237799
           p03           9.833538           7.527270  23.453084
           p04        1517.849103         715.065836  52.889531

coverage             
planner     asp1-4h asp1-4h-undo
domain                          
acrobatics      0.5          0.5

382.59 181.31 0.526098434355315
Average decrease in solve times by 19.04%


### Blocksworld

In [254]:
df_c, p1_time, p2_time, improvement = stats_kb("blocksworld-ipc08", 'asp1-4h', 'asp1-4h-undo')
display(df_c)
print(p1_time, p2_time, improvement)
print(f"Average decrease in solve times by {improvement}%")

time_asp1-4h  time_asp1-4h-undo       diff
domain            instance                                            
blocksworld-ipc08 p01          10.070023           9.052898  10.100530
                  p02          10.073025           9.428407   6.399446
                  p03          14.518322          12.331279  15.064020
                  p04          63.505597          58.968700   7.144089
                  p05          43.004140          36.590771  14.913376
                  p06          28.152336          19.102322  32.146584
                  p07          53.570665          41.830169  21.915904
                  p08          56.370559          42.052182  25.400452
                  p09          12.862129           9.276320  27.878817
                  p10          25.035971          17.433080  30.367870

coverage             
planner             asp1-4h asp1-4h-undo
domain                                  
blocksworld-ipc08  0.333333     0.333333

31.72 25.61 19.13
Average decrease in solve times by 19.13%


### First Responders

Solved extra

In [255]:
df_c, p1_time, p2_time, improvement = stats_kb("first-responders-ipc08", 'asp1-4h', 'asp1-4h-undo')
display(df_c)
print(p1_time, p2_time, improvement)
print(f"Average decrease in solve times by {improvement}%")

time_asp1-4h  time_asp1-4h-undo        diff
domain                 instance                                             
first-responders-ipc08 p02           1.196759           1.349788  -12.786896
                       p03           1.217278           1.181084    2.973311
                       p04           1.298381           1.356813   -4.500407
                       p05           1.445392           1.320012    8.674484
                       p06           1.729716           1.551979   10.275553
                       p07           2.267339           1.925842   15.061597
                       p08           5.130778           3.418421   33.374211
                       p10         121.300851          59.733867   50.755608
                       p101          3.174875           7.136807 -124.790176
                       p102          6.964646           8.261159  -18.615640
                       p103      11929.327975        3966.431583   66.750587
                       p12           1.220395           1.107219    9.273662
                       p13           2.573410           1.835960   28.656541
                       p14           2.633469           1.930758   26.683864
                       p17        1494.053291         797.122869   46.646959
                       p18          85.028978          63.894878   24.855174
                       p31           1.347685           1.155151   14.286297
                       p32           2.721870           2.265655   16.761100
                       p37        4553.915309        7646.683069  -67.914477
                       p38        7784.632169        7106.533897    8.710730
                       p41           1.406776           1.109076   21.161844
                       p42           1.926374           1.844441    4.253253
                       p43           2.527400           2.392091    5.353695
                       p44         663.541491         581.523619   12.360624
                       p46          49.337028          59.999068  -21.610624
                       p51           1.399852           1.385151    1.050125
                       p52           2.396648           1.970721   17.771786
                       p53           5.326943           3.555664   33.251323
                       p54         623.631562         175.908951   71.792808
                       p61           2.350949           2.123349    9.681196
                       p62           2.992923           2.226566   25.605641
                       p63         164.578877          95.611257   41.905511
                       p64         595.151530         380.420056   36.080135
                       p65         897.781132         422.923153   52.892399
                       p71           1.691038           1.856200   -9.766882
                       p72           4.384648           3.276245   25.279169
                       p73          61.074907          51.488596   15.695990
                       p74                NaN        8416.700021         NaN
                       p75       11435.351403        5582.029581   51.186200
                       p78         136.574632          89.285203   34.625339
                       p81           2.542023           2.885543  -13.513672
                       p82           5.150716           4.285882   16.790552
                       p84                NaN       12434.478776         NaN
                       p91           9.065341          10.512816  -15.967135
                       p92          21.656618          25.521921  -17.848136
                       p93         871.033020         799.500082    8.212426

coverage             
planner                 asp1-4h asp1-4h-undo
domain                                      
first-responders-ipc08  0.60274     0.630137

944.68 1061.54 12.3
Average decrease in solve times by 12.3%


### Tireworld Triangle

Solved two extra

In [260]:
df_c, p1_time, p2_time, improvement = stats_kb("triangle-tireworld", 'asp1-4h', 'asp1-4h-kb')
print(p1_time, p2_time, improvement)
print(f"Average decrease in solve times by {improvement}%")

time_asp1-4h  time_asp1-4h-kb       diff
domain             instance                                          
triangle-tireworld p01           1.537400         1.179282  23.293723
                   p02           6.988352         3.914404  43.986743
                   p03         612.098800        28.698677  95.311431
                   p04       11263.077813       175.680043  98.440213

2970.93 52.37 65.26
Average decrease in solve times by 65.26%


### Islands

In [261]:
df_c, p1_time, p2_time, improvement = stats_kb("islands", 'asp1-4h', 'asp1-4h-undo')
display(df_c)
print(p1_time, p2_time, improvement)
print(f"Average decrease in solve times by {improvement}%")

time_asp1-4h  time_asp1-4h-undo       diff
domain  instance                                            
islands p01           1.163689           1.160686   0.258028
        p02           1.194491           1.251924  -4.808114
        p03           1.141910           1.222344  -7.043852
        p04           1.297221           1.335027  -2.914334
        p05           1.311996           1.334496  -1.714935
        p06           1.295695           1.354173  -4.513225
        p07           1.370086           1.562100 -14.014752
        p08           1.372773           1.440839  -4.958332
        p09           1.446835           1.544890  -6.777151
        p10           1.530776           1.550399  -1.281852
        p11           1.515324           1.784365 -17.754669
        p12           1.726644           1.856358  -7.512498
        p13           1.316649           1.402338  -6.508153
        p14           1.489100           1.695987 -13.893484
        p15           1.536003           1.941025 -26.368590
        p16           2.216019           2.065818   6.777936
        p17           2.645698           2.834942  -7.152891
        p18           2.513790           3.166131 -25.950501
        p19           2.772826           3.738611 -34.830315
        p20           3.532140           4.078231 -15.460641
        p21           4.161130           4.979042 -19.656032
        p22           4.144279           5.750010 -38.745731
        p23           5.155720           5.956963 -15.540861
        p24           5.575879           6.205719 -11.295799
        p25           1.705148           1.855833  -8.837026
        p26           2.725590           3.125557 -14.674523
        p27           4.592115           4.943574  -7.653511
        p28           6.612907           5.905874  10.691723
        p29           7.656748           8.676252 -13.315094
        p30           9.973182          10.821921  -8.510205
        p31           9.927314          15.602609 -57.168480
        p32          13.628599          17.477564 -28.241829
        p33          14.054298          21.804885 -55.147446
        p34          17.026465          25.540700 -50.005882
        p35          22.649359          34.033461 -50.262357
        p36          26.222870          33.518234 -27.820620
        p37           3.667669           2.947576  19.633531
        p38           7.704491           7.084507   8.047050
        p39          12.841346          13.144307  -2.359263
        p40          17.560445          21.444436 -22.117843
        p41          26.384054          28.924293  -9.627933
        p42          32.390635          37.648954 -16.234072
        p43          43.138279          52.131936 -20.848437
        p44          48.839811          64.567227 -32.202042
        p45          65.976707          78.437971 -18.887368
        p46          70.164008         101.119403 -44.118625
        p47          85.250079         113.638870 -33.300604
        p48         102.456026         142.590424 -39.172315
        p49           8.278214           9.939419 -20.067190
        p50          21.523726          21.128977   1.834020
        p51          37.119735          37.075828   0.118284
        p52          51.710278          62.164874 -20.217636
        p53          73.406928          82.495296 -12.380804
        p54         100.735834         119.010786 -18.141460
        p55         128.706453         159.577315 -23.985481
        p56         171.305644         207.035303 -20.857257
        p57         210.722836         262.436919 -24.541281
        p58         244.391377         341.451306 -39.714957
        p59         285.252008         404.996159 -41.978372
        p60         338.898155         471.501890 -39.127901

coverage             
planner  asp1-4h asp1-4h-undo
domain                       
islands      1.0          1.0

39.64 51.45 -17.71
Average decrease in solve times by -17.71%


### Miner

In [262]:
df_c, p1_time, p2_time, improvement = stats_kb("miner", 'asp1-4h', 'asp1-4h-kb')
display(df_c)
print(p1_time, p2_time, improvement)
print(f"Average decrease in solve times by {improvement}%")

time_asp1-4h  time_asp1-4h-kb  time_asp1-4h-undo       diff
domain instance                                                             
miner  p01          18.707134        17.892042          17.284011   4.357119
       p02          16.076967        15.135138          14.574727   5.858253
       p03          20.045592        19.333486          19.568227   3.552429
       p04          74.401413        79.408012          78.042042  -6.729172
       p05          69.450128        73.652528          69.467090  -6.050960
       p07         157.143603       161.137660         162.451868  -2.541661
       p08         118.538299       120.444250         127.198438  -1.607877
       p09         157.851991       168.399388         171.036193  -6.681828
       p10         110.130568       120.344347         126.592267  -9.274246
       p11         124.578368       121.832011         126.436822   2.204522
       p12          74.528633        74.514239          74.822833   0.019313
       p13          85.167154        90.518674          85.109152  -6.283549
       p14         169.395533       207.999971         205.705968 -22.789526
       p15         206.021501       219.965519         228.906555  -6.768235
       p16         209.792574       248.858218         244.157353 -18.621081
       p17         239.061168       281.804068         302.228880 -17.879483
       p18         183.488007       198.115265         218.238561  -7.971779
       p19         425.613644       501.847758         548.400071 -17.911577
       p20         286.250793       314.372850         343.119816  -9.824272
       p21         396.612827       432.441897         444.192076  -9.033765
       p22         651.669466       652.570804         681.115875  -0.138312
       p23         413.702961       480.140382         484.909949 -16.059208
       p24         597.700828       680.497114         682.064190 -13.852463
       p25         252.617577       280.587653         279.173330 -11.072102
       p26         255.806790       292.308566         301.681196 -14.269275
       p27         486.642464       516.333685         545.955184  -6.101239
       p28         432.562960       472.585662         489.540168  -9.252457
       p29         311.423296       334.872964         339.473163  -7.529837
       p30         214.990530       249.981798         253.948800 -16.275725
       p31         236.404090       258.505048         272.293343  -9.348805
       p32        1070.601287      1049.961388         997.217404   1.927879
       p33         655.832682       731.728057         713.800341 -11.572369
       p34         393.940576       378.906166         477.492995   3.816416
       p35         420.369528       418.565670         532.035360   0.429112
       p36         375.740863       335.636988         447.867767  10.673280
       p37         490.588972       424.421155         570.151040  13.487424
       p38         977.765567       745.654043         996.632428  23.738975
       p39        1170.402049       913.761282        1111.193871  21.927573
       p40         765.122059       615.374338         769.436869  19.571743
       p41         696.243409       576.315599         739.832118  17.224983
       p42         613.307739       514.516238         693.031448  16.107982
       p43         819.248599       668.474200         665.017792  18.403986
       p44         681.324398       587.870381         576.004411  13.716523
       p46        1549.705391      1109.485809        1124.254201  28.406663
       p50        1326.003717       818.045814        1108.990192  38.307427

coverage                        
planner  asp1-4h asp1-4h-kb asp1-4h-undo
domain                                  
miner       0.96       0.94          0.9

422.28 390.56 -0.48
Average decrease in solve times by -0.48%


### Spiky Tireworld


In [263]:
df_c, p1_time, p2_time, improvement = stats_kb("spiky-tireworld", 'asp1-4h', 'asp1-4h-undo')
display(df_c)
print(p1_time, p2_time, improvement )
print(f"Average decrease in solve times by {improvement}%")

time_asp1-4h  time_asp1-4h-undo       diff
domain          instance                                            
spiky-tireworld p01         222.185609          73.735810  66.813418
                p02         394.077962         114.040968  71.061318
                p03         546.400103         151.657801  72.244185
                p04          60.646958          25.660017  57.689523
                p05         776.226829         251.125232  67.647958
                p06        1127.264979         310.496727  72.455746
                p07        1258.458311         384.827293  69.420736
                p08        1613.661388         446.857107  72.307876
                p09        1834.835714         556.143481  69.689740
                p10        2096.901949         598.881418  71.439703

coverage             
planner           asp1-4h asp1-4h-undo
domain                                
spiky-tireworld  0.909091          1.0

993.07 291.34 69.08
Average decrease in solve times by 69.08%


### Tireworld Truck

In [264]:
df_c, p1_time, p2_time, improvement = stats_kb("tireworld-truck", 'asp1-4h', 'asp1-4h-undo')
display(df_c)
print(p1_time, p2_time, improvement)
print(f"Average decrease in solve times by {improvement}%")

time_asp1-4h  time_asp1-4h-undo       diff
domain          instance                                            
tireworld-truck p01           1.640760           1.764090  -7.516679
                p02           1.454048           1.528061  -5.090102
                p03           1.896543           1.657823  12.587101
                p04           2.543349           2.373627   6.673163
                p05           3.630763           3.413070   5.995806
...                                ...                ...        ...
                p71         210.946242         125.632983  40.443128
                p72         965.528965         522.881821  45.845040
                p73         215.594749         167.694391  22.217776
                p74          81.644215          71.099423  12.915541
                p75          39.910197          36.534682   8.457777

[74 rows x 3 columns]

coverage             
planner          asp1-4h asp1-4h-undo
domain                               
tireworld-truck      1.0          1.0

47.98 33.36 17.26
Average decrease in solve times by 17.26%
